# Open Data Toronto

Snippet taken from the ["For Developers" tab](https://open.toronto.ca/dataset/311-service-requests-customer-initiated/)

In [1]:
import requests

# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "311-service-requests-customer-initiated"}
package = requests.get(url, params = params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

       # To get metadata for non datastore_active resources:
       if not resource["datastore_active"]:
           url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
           resource_metadata = requests.get(url).json()
           print(resource_metadata)
           # From here, you can use the "url" attribute to download this file


{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=resource_show', 'success': True, 'result': {'cache_last_updated': None, 'cache_url': None, 'created': '2019-07-23T16:30:17.170264', 'datastore_active': False, 'format': 'XLSX', 'hash': '', 'id': 'bb11ece8-a2b3-401a-92c0-fa115f867fff', 'is_datastore_cache_file': False, 'is_preview': False, 'last_modified': '2019-07-23T16:30:17.086581', 'metadata_modified': '2022-03-10T16:08:57.037006', 'mimetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'mimetype_inner': None, 'name': '311-service-requests-readme', 'package_id': '2e54bc0e-4399-4076-b717-351df5918ae7', 'position': 0, 'resource_type': None, 'revision_id': 'c93dcb0a-5b4d-4048-927b-143ab63e6f8c', 'size': 13487, 'state': 'active', 'url': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2e54bc0e-4399-4076-b717-351df5918ae7/resource/bb11ece8-a2b3-401a-92c0-fa115f867fff/download/311-service-requests-readme.xlsx', 'url_type'